# Classification: Classify text into categories or labels using chat models with structured outputs.

Gắn thẻ là gắn nhãn cho các tài liệu vào các lớp...
## Content
1. Setup
2. Finer control
3. Going deeper

# 1. Setup

Trong dự án này sử dụng mô hình LLM là Groq

In [ ]:
%pip install --upgrade --quiet langchain-core

In [ ]:
%pip install -qU "langchain[groq]"

In [ ]:
import os
import getpass

from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

In [ ]:
# Groq:
# GROQ_API_KEY

In [ ]:
if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

llm = init_chat_model("llama3-8b-8192", model_provider="groq")

In [ ]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
        Extract the desired information from the following passage.

        Only extract the properties mentioned in the 'Classification' function.

        Passage:
        {input}
    """
)

In [ ]:
%pip install -U langchain-openai
%export OPENAI_API_KEY="your-api-key"

In [ ]:
class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(description="How aggressive the text is on a scale from 1 to 10")
    language: str = Field(description="The language the text is written in")


llm = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(Classification)


In [ ]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)
print(response)

In [ ]:
# Nếu chúng ta muốn đầu ra từ điển, chúng ta chỉ cần gọi .model_dump()
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

response.model_dump()

# 2. Finer control

Định nghĩa lược đồ là rất quan trọng, giúp chúng ta kiểm soát đầu ra tốt hơn.

In [ ]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=['happy', "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )

    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )    

In [ ]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
        Extract the desired information from the following passage.

        Only extract the properties mentioned in the 'Classification' function.

        Passage:
        {input}
    """
)

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(Classification)

Bây giờ các câu trả lời sẽ bị hạn chế theo cách chúng ta mong đợi!

In [ ]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

In [ ]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

In [ ]:
inp = "Weather is ok here, I can go outside without much more than a coat"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

# 3. Going deeper

- You can use the metadata tagger document transformer to extract metadata from a LangChain Document.
- This covers the same basic functionality as the tagging chain, only applied to a LangChain Document.